## Checking calculations

Notebook F.A.O. Jack & Chris

Note: I show these for cycling here but it's the same calculations for running

In [1]:
import pandas as pd
google = pd.read_excel('data_to_analyse.xlsx',sheet_name='GOOGLE_2023')
census = pd.read_excel('data_to_analyse.xlsx',sheet_name='CENSUS_2022')

Google data taken from all "real" constituencies. the Dublin provided google file doesn't include north and south dublin county.

In [2]:
google['POPULATION'].sum()

google.head(2)

,ENG_NAME_VALUE,NAME,POPULATION,AUTOMOBILE_trips,AUTOMOBILE_full_distance_km,AUTOMOBILE_gpc_distance_km,AUTOMOBILE_full_co2e_tons,AUTOMOBILE_gpc_co2e_tons,BUS_trips,BUS_full_distance_km,...,RAIL_trips,RAIL_full_distance_km,RAIL_gpc_distance_km,RAIL_full_co2e_tons,RAIL_gpc_co2e_tons,TRAM_trips,TRAM_full_distance_km,TRAM_gpc_distance_km,TRAM_full_co2e_tons,TRAM_gpc_co2e_tons
0,dublin-bay-north,Dublin Bay North,150114,40864985,479492825,271108155.0,87147.8673,49273.9334,3973751,29384540,...,2432793,26915828,13457914.0,0,0,0,0,0.0,0,0
1,dublin-bay-south,Dublin Bay South,123593,34187328,540945630,282679363.0,98316.9206,51377.0016,1914481,5749581,...,4621146,70094390,35047195.0,0,0,523089,1284019,1284019.0,0,0


Census data here is already aggregated from the small area maps data so it's just a single row. I checked and the population figure is exactly correct, which means our data is coming in right

In [3]:
census.head(3)

,Unnamed: 0,Total population,On foot - Total,Bicycle - Total,"Bus, minibus or coach - Total","Train, DART or LUAS - Total",Motorcycle or scooter - Total,Car driver - Total,Car passenger - Total,Van - Total,Other (incl. lorry) - Total,Work mainly at or from home - Total,Not stated - Total
0,CENSUS 2022,1458154,180789,64387,127159,61401,5118,272279,124443,21137,1837,89359,98530


### Google data

Let's look at any one mode of transport, e.g. cycling

In [4]:
cyc = [col for col in google.columns if 'CYCLING' in col]

In [5]:
cyc

['CYCLING_trips',
 'CYCLING_full_distance_km',
 'CYCLING_gpc_distance_km',
 'CYCLING_full_co2e_tons',
 'CYCLING_gpc_co2e_tons']

It is the GPC stats that we use.

In [6]:
google[cyc].sum()

CYCLING_trips                30512997.0
CYCLING_full_distance_km    130597099.0
CYCLING_gpc_distance_km      82458101.0
CYCLING_full_co2e_tons              0.0
CYCLING_gpc_co2e_tons               0.0
dtype: float64

so that is 30,512,997 (30m) cycling trips in the year

Can we equate these two datsets together? Let's try. Recall how many people said they bike to work

In [7]:
census['Bicycle - Total'].iloc[0]

64387

then assuming 254 working days a year, 2 journeys a day...

In [8]:
census['Bicycle - Total'].iloc[0]*254*2

32708596

Over half (55pc) of office employees are now spending between two and three days at the workplace each week, according to Dublin Chamber’s latest Quarterly Business Outlook Survey.

So a good average might be to assume average person goes into work 3.5 (?) days a week

In [9]:
census['Bicycle - Total'].iloc[0]*254*2*(3.5/5)

22896017.2

that is 22m: over two thirds of all trips captured by google

Notes:
- Census is self-reported: people might say they cycle but actually don't
- The question asks what form people USUALLY take, you might usually cycle in the summer etc

## 1: CO2 saved [Google vs Census]

The CO2 saved by not driving.



### 1.1 Google

Google tracks the CO2 emissions automatically.

Logic: "If every person who was cycling was magically driving instead, how much CO2 would they generate?"

In [10]:
google['CYCLING_trips'].sum()

30512997

In [11]:
auto = [col for col in google.columns if 'AUTOMOBILE' in col]

In [12]:
google[auto].sum()

AUTOMOBILE_trips               4.778163e+08
AUTOMOBILE_full_distance_km    6.858249e+09
AUTOMOBILE_gpc_distance_km     3.731595e+09
AUTOMOBILE_full_co2e_tons      1.246487e+06
AUTOMOBILE_gpc_co2e_tons       6.782178e+05
dtype: float64

Method: assume each bike trip is now an automobile trip - find out how much emissions this would generate

In [13]:
emissions_per_AUTOMOBILE_trip = google['AUTOMOBILE_gpc_co2e_tons'].sum()/google['AUTOMOBILE_trips'].sum()

emissions = google['CYCLING_trips'].sum()*emissions_per_AUTOMOBILE_trip

emissions

43310.48626089091

Cyclists are saving us 43,310 tonnes of CO2 a year by not driving

### 1.2 Census commuting data

We need to calculate the emissions ourselves.

Logic: "If every person who was commuting was instead commuting by car, how much CO2 would they generate?"

Keeping our earlier assumptions - 254 working days a year, 2 journeys per day, 3.5 days a week.

In [14]:
census['Bicycle - Total'].iloc[0]*254*2*(3.5/5)

22896017.2

TomTom data: car uses 95g CO2/km = 95 x 10^-6 Tonnes CO2/km

Assume average commute: 10km

In [15]:
emissions_saved_census = 254*2*(3.5/5)*(95/1000000)*10

In [16]:
census['Bicycle - Total'].iloc[0]*emissions_saved_census

21751.21634

Cyclists are saving us 21,751.21 tonnes of CO2 a year by not driving. This figure is less trustworthy but is correctly under the google value.

#### 2. Total emissions [Google]

Total automobile emissions

In [17]:
google['AUTOMOBILE_gpc_co2e_tons'].sum()

678217.7682999999

678,217 Tonnes of Carbon generated by autombiles

In [18]:
google['BUS_gpc_co2e_tons'].sum()

95849.3202

95,000 Tonnes of Carbon generated by buses

### 3. Time saved [Census]

TomTom: 

- 10km commute 5 days a week
- 295h spent driving
- 158h lost due to congestion in dublin city
- 90h lost in dublin metro area

Let's take the average of city and metro

We are also assuming 3.5 days a week to be more conservative

In [19]:
time_saved_per_person = 0.5*(158+90)*(3.5/5)

time_saved_per_person

86.8

One person saves this 86.8 hours a year by not sitting in traffic. what does everybody save?

In [20]:
hours_saved = census['Bicycle - Total'].iloc[0]*time_saved_per_person

converting to years

In [21]:
years_saved = hours_saved/(24*365)

years_saved

637.989908675799

Over a year Dublin's cyclists save 637 years of time they would otherwise spend sitting in traffic. 

### 4. Fuel savings [Census]

TomTom
- 10km commute 5 days a week = 
- 695/628 euro petrol (city vs metro area)
- 599/548 euro diesel (city vs metro area)



Let's be conservative and assume diesel and our earlier 3.5 days in office a week

In [22]:
fuel_savings_perperson = 0.5*(599+548)*(3.5/5)

In [23]:
fuel_savings_perperson

401.45

The average cyclist saves 401 euro a year by not commuting

In [24]:
fuel_savings_dublin = census['Bicycle - Total'].iloc[0]*fuel_savings_perperson

In [25]:
fuel_savings_dublin

25848161.15

Dubliners save 25 million euro a year by cycling instead of driving

### 5. Premature deaths avoided [Census]

The number of premature deaths averted this year by walking or cycling to work. 

- For cycling, biking to work reduces the risk of premature death by 41%. 

- Walking 150 mins a week reduces this by 16%. 

- The risk of premature death between 30-70 years was estimated at 10%.

Logic: "If our cyclists were driving instead, how many of them would die prematurely this year?"

risk of premature death in any year = 10% / 40 years

In [26]:
0.1*0.1/40

0.00025000000000000006

In [27]:
premature_deaths_if_driving = census['Bicycle - Total'].iloc[0]*0.1/40

In [28]:
premature_deaths_if_driving

160.96750000000003

In [29]:
premature_deaths_prevented = premature_deaths_if_driving*0.41

In [30]:
premature_deaths_prevented

65.99667500000001

66 deaths prevented this year alone through cycling to work

#### 6. Cost of congestion

2022 study put the cost of congestion as 336m in the greater dublin area (GDA), pop 2,073,459

Logic: 

- work out what fraction of the GDA is Dublin
- find the ratio of total car + bus trips to the cost of congestion 
- based on this ratio, estimate how much extra cycling or walking trips would add to this congestion

This way underestimates the number as the relationship between people and traffic is not linear. The document I have around cost of congestion details their methodology but it's not really one we can plug these numbers into.

In [31]:
gda_fraction = google['POPULATION'].sum()/2073459

In [32]:
cost = 336000000*gda_fraction

247.63m cost of congestion in our dublin map

In [33]:
total_trips = google['AUTOMOBILE_trips'].sum()+google['BUS_trips'].sum()

total_trips



512234697

In [34]:
cost_per_trip = cost/total_trips

In [35]:
cost_per_trip

0.4834317104213985

In [36]:
google['CYCLING_trips'].sum()*cost_per_trip/1000000

14.750950329793001

In [37]:
google['ON FOOT_trips'].sum()*cost_per_trip/1000000

144.0280321276569

So that is 144m saved by walking and 14m by cycling.

Surely it should be a lot more!